# Athena : Grid-based code for astrophysical magnetohydrodynamics (MHD)

In this notebook we will discover how to use (and potential extend) a community code called **athena**. You can use the ASCL service to find out more about such a code (although google can sometimes be more effective):
http://ascl.net/athena should redirect you to the relevant ASCL entry or entries for that code.

In fact, you will discover two groups came up with this name.

You will need to use the terminal as well in this notebook.  An in-depth discussion of MHD codes is beyond our current scope, but we have to recall that python is not always the best code to implement codes that need a large number of computations.  An example of a related problem that was solved in a notebook using python, is the heat equation using the Crank-Nickelson method, and is described [here](http://nbviewer.jupyter.org/github/waltherg/notebooks/blob/master/2013-12-03-Crank_Nicolson.ipynb).

## 1. downloading

Following the code site we will find a link on https://trac.princeton.edu/Athena/wiki/AthenaDocsDownLd or you can also use that link in a terminal and use the command
```
   wget http://www.astro.princeton.edu/~jstone/downloads/athena/athena4.2.tar.gz
   tar zxf athena4.2.tar.gz
```
or
``` 
   curl http://www.astro.princeton.edu/~jstone/downloads/athena/athena4.2.tar.gz | tar zxf -
```
depending on which program you have, or what method you prefer. Either way, you will now have the athena directory.

If there are problems, a private github repo on https://github.com/astroumd/athena4.2 has been maintained with a smaller number of dependencies, including some example data that can be directly used for analysis. Contact the owner(s) for a snapshot if you need one.

## 2. configuring, compiling and testing

The athena program is written in the C language, so it must be compiled first. python is an interpreted language, so it does not need this step. compiled languages are generally many times (sometimes 10 or 100 or even 1000) faster than their equivalent code in interpreted languages. However, developing in a language like python can often be a lot faster than in a compiled language.

```
   cd athena-4.2
   ls
   autoconf
```
this is where you might see a **command not found** error message. That's a sign of trouble. It means your system does not have some basic tools installed, buy [autoconf](https://github.com/edrosten/autoconf_tutorial) is one needed for athena. Either you install this tool, e.g.
```
   sudo apt-get install autoconf                     # example on ubuntu
```

or grab the private repo version as discussed before. Assuming we have this problem solved, we continue with
```
   ./configure 
   ...
   
Your Athena distribution has now been configured:

--- Physics Options ---

Problem:                 linear_wave
Gas properties:          MHD
Equation of State:       ADIABATIC
Coordinate System:       CARTESIAN
Advected scalar fields:  0
Self-gravity:            OFF
Resistivity:             OFF
Viscosity:               OFF
Thermal conduction:      OFF
Particles:               OFF
Special Relativity:      OFF

--- Algorithm Options ----

Spatial Order:           3 (THIRD_ORDER_CHAR)
Flux:                    roe
unsplit integrator:      ctu
Precision:               DOUBLE_PREC
Compiler options:        -O3
Ghost cell output:       OFF
Parallel modes: MPI      OFF
H-correction:            OFF
FFT:                     OFF
Shearing-box:            OFF
FARGO:                   OFF
Super timestepping:      OFF
Static Mesh Refinement:  OFF
first-order flux corr:   OFF
ROTATING_FRAME:          OFF
L1_INFLOW:               OFF

```
   Now your athena has been set up for a particular problem (the linear wave) in 1D. If you want to learn more about the options to configure, use
   
```
    ./configure --help
```
Compilation is a simple
```
    make all
```
after this there will be a binary in the direcetory **bin/athena**. There is a standard simple test (in 1D) that acts like a benchmark to see how fast the code is. This also allows you very easily to test out if there are certain compile flags that will optimize the code even better.
```
    make test
(cd tst/1D-mhd; ./run.test)
zone-cycles/cpu-second = 9.116444e+05
zone-cycles/wall-second = 8.969274e+05
L1 norm for density: 2.665900e-11
```   
Inspecting the **run.test** script will be instructive to see how you run this program.

**Q1:**  how do you compile a version of the program with the "-g" debug option?  The code should run a little slower. How about "-g -O0", meaning with no optimization (sometimes you need to shut up the compiler from optimizing even in debug mode).

## 3. running 

Running the code now involves....

## 4. analysis

This assumes you have data files ... in your current directory.